# Linear Programming, Optimization
**Fundamental theorem of linear programming**
"... Any linear program either is infeasible, is unbounded, or has an optimal solution with a finite objective value. In each case, [the algorithm] SIMPLEX acts appropriately." I.e. simplex will return an optimal solution or 'unbounded' or 'infeasible.' 
- *Introduction to Algorithms*, Third Edition, Cormen, Leirson, Rivest and Stein, 2009, MIT, Cambridge, Massachussetts, pg. 891

*Note:*
- "The Simplex algorithm does not run in polynomial time in the wors case, but it is fairly efficient and widely used in practice." (Ibid, 864)

😎 **In other words:** *If you can formulate a problem as a system of linear equations, and you have more than two variables (dimensions) then the simplex algorithm is probably your best bet to solving it.*

## Contents:
- pending completion

## Basic Example: Optimizing farming profit
Suppose a farmer has 240 acres of land. 
- they can farm corn for a profit of \$40/acre and time cost of 2hr/acre
- they can farm oats for a profit of \$30/acre and time cost of 1hr/acre
- they only have 320 farmable hours before harvest is due

How many acres of each should they plant to maximize profits?

### We formulate the word problem as a linear program

**Objective function:** profits `p = 40*c + 30*o`

subject to:

**Constraint 1:** `c + o <= 240` total acres farmed 

**Constraint 2:** `2c + o <= 320` total hours farmed

**Constraint 3:** `c, o >= 0` because they can't farm negative acres 

### We can graph the system of linear equations and constraints

In [39]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
from scipy.optimize import linprog

### Graph the solution space for the number of farmable acres

In [27]:
# notice the non-negativity constraint (i.e. constraint 3) is baked into the code here
acres = [a for a in range(0, 240)] 
    
acres_trace = go.Scatter(
    x = acres,
    # for y <= 240 - x
    y = [240 - a for a in acres], 
    fill = 'tozeroy',
    name = 'farmable acres; constraint 1'.title(),
    
)
data = [acres_trace]
layout = go.Layout(
    title = 'Feasible Solution Area',
    xaxis = {'title': 'acres of corn'},
    yaxis = {'title': 'acres of oats'},
    showlegend = True,
    legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig = go.Figure(data = data, layout = layout)
fig

**Constraint 1: `c + o <= 240` total acres farmed**
- The above graph visualizes the constraint relationship between acres farmed for corn and for oats.
- if you farm 240 acres of corn, you will have no acres left for oats and vice versa.

### We add the constraint of farmable hours

In [28]:
hours = [h for h in range(0, 320)]
hours_trace = go.Scatter(
    x = hours,
    # y <= 320 - 2x
    y = [320 - 2*h for h in hours if 320 - 2*h >= 0], 
        #we add the conditional to the list comprehension to satisfy the non-negativity constraint
    fill = 'tozeroy',
    name = 'farmable hours; constraint 2'.title(),
)

data = [acres_trace, hours_trace]
layout = go.Layout(
    title = 'Feasible Solution Area',
    xaxis = {'title': 'acres of corn'},
    yaxis = {'title': 'acres of oats'},
    showlegend = True,
    legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig = go.Figure(data = data, layout = layout)
fig

**Constraint 2:** `2c + o <= 320` total hours farmed
- the above graph shows the constrain relationship between hours farmed for corn and oats
- if you use all 320 hours farming corn you won't have any hours left to farm oats

Notice the intersection of the feasability spaces is a solution space to the system of equations resulting from combining both constraints

### Now we add the objective function (aka the profit function) to the system 
- `p = 40c + 30o` substitutting 
    - corn for `x` and 
    - oats for `y` 
- we can isolate `y` for a version of the familiar **y = mx + b** 
    - `p = 40x + 30y` --> `p - 40 = 30y` --> `y = (p/30) - (40/30)x` 



In [29]:
# declare constrained values of plantable corn acres
corn_acres = [a for a in range(240)]
# declare constrained values of plantable oats acres
oats_acres = [a for a in range(240)]
# profit as a 2D, 240x240 matrix with values p = 40c + 30o
# init the profit matrix as zeros
profit = np.zeros((240,240))

# for every possible number of corn acres planted
for i_corn in corn_acres:
    #for every possible number of oats acres planted
    for j_oat in oats_acres:
        # the profit at that combo; p = 40c + 30o
        profit[i_corn][j_oat] = 40*i_corn + 30*j_oat

In [38]:
fig.add_contour(
    # add the profit as a z value (need to feed it transposed to match up the axes and coords)
    z = profit.T,
    colorscale = 'Greens',
    #contours_coloring = 'lines',
    #line_width = 2,
    name = 'Profit Margins',
    contours = dict(
        coloring = 'heatmap',
        showlabels = True,
        start = 0,
        end = np.max(profit),
        # 500 increments
        size = 500
    ),
)
fig

**The interactive plot gives us an intuitive sense of how to maximize the profit, given the constraints**
- we may have expected corn to be more profitable, but if you went "all in on corn." you'd run out of time after planting the 160th acre and only expect a profit of 6,400 and 80 of your acres would go unused
- You might be better off going "all in on oats," expecting a profit of 7,200 and only having to work 240 hours (you get 80 hours left to do other stuff).

**the most profitable solution** is 80 acres of corn (3,200 profit), 160 of oats (4,800 profit), for a combined profit of 8,000 
    - (and perhaps lower risk that your entire yield would be wiped out by a condition that affects one crop but not the other)

## General Algorithmic Solution: `scipy.optimize.linprog`
- in general, you may have more than two constraints, more than two profit variables, etc. Anything beyond three dimensions won't be visualizable, but luckily math has a way to solve these functions for us.
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html


In [46]:
# "cost function" to minimize (just the negative of the profit to maximize)
# maximize p = 40c + 30c
p = [-40, -30]

# constraints in the "design matrix"; A
constraints = [
    # c + o <= 240 ; acres
    [1, 1],
    # 2c + o <= 320; hours
    [2, 1]
]

# bounds of the constraints
upper_bounds = [240, 320]

# bounds of corn and oats
corn_bounds = (0,240)
oats_bounds = (0,240)

# results
res = linprog(
    c = p,
    A_ub = constraints,
    b_ub = upper_bounds,
    bounds = [corn_bounds, oats_bounds]
)
res

     con: array([], dtype=float64)
     fun: -7999.999996240393
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([1.12949635e-07, 1.50061396e-07])
  status: 0
 success: True
       x: array([ 79.99999996, 159.99999992])

## here is the optimal value for the linear program

In [55]:
print(f'Optimal Profit: ${round(-res.fun):,}')
print(f'Optimal Corn Acres to Plant: {round(res.x[0]):,}')
print(f'Optimal Oat Acres to Plant: {round(res.x[1]):,}')

Optimal Profit: $8,000
Optimal Corn Acres to Plant: 80
Optimal Oat Acres to Plant: 160
